In [22]:
from pref_voting.profiles_with_ties import *
from pref_voting.voting_methods import *

from pref_voting.generate_profiles import *
from pref_voting.generate_weighted_majority_graphs import *

from pref_voting.utility_profiles import UtilityProfile

from pref_voting.generate_utility_profiles import *
from pref_voting.generate_utility_profiles import *
from pref_voting.utility_methods import *

from pref_voting.spatial_profiles import SpatialProfile
from pref_voting.generate_spatial_profiles import *
from pref_voting.utility_functions import *
from pref_voting.utility_methods import *

from tqdm.notebook import tqdm


In [28]:
num_cands = 10
num_voters = 100
num_dims = 2

cand_cov = generate_covariance(num_dims, 1, 0.0)
voter_cov = generate_covariance(num_dims, 1, 0.0)

num_trials = 10000

vms = [
    plurality, 
    plurality_with_runoff,
    instant_runoff,
    borda,
    coombs,
    blacks, 
    minimax, 
    stable_voting,
    beat_path_Floyd_Warshall, 
    ranked_pairs_zt]

social_utilities_candidates = list()
social_utilities_vm_winners = {vm.name: list() for vm in vms}
social_utilities_util_winners = list()

for t in tqdm(range(num_trials)): 
    sp = generate_spatial_profile(num_cands, num_voters, num_dims, cand_cov = cand_cov, voter_cov = voter_cov)

    uprof = sp.to_utility_profile(utility_function=linear_utility).normalize()
    prof = uprof.to_ranking_profile().to_linear_profile()

    avg_util = uprof.avg_utility_function()


    util_ws = sum_utilitarian_ws(uprof)

    for w in util_ws: 
        social_utilities_util_winners.append(avg_util(w))

    ws_dict = {vm.name: vm(prof) for vm in vms}

    for vm in vms:
        for w in ws_dict[vm.name]:
            social_utilities_vm_winners[vm.name].append(avg_util(w))

    for c in prof.candidates: 
        social_utilities_candidates.append(avg_util(c))


avg_social_utilities_util_winners = np.average(social_utilities_util_winners)
avg_social_utilities_vm_winners = {vm.name: np.average(social_utilities_vm_winners[vm.name]) for vm in vms}
avg_social_utilities_candidates = np.average(social_utilities_candidates)

soc_util_efficiency = {vm.name: (avg_social_utilities_vm_winners[vm.name] - avg_social_utilities_candidates) / (avg_social_utilities_util_winners - avg_social_utilities_candidates) for vm in vms}

for vm in vms: 
    print(f"Social Utility Efficiency for {vm.name}: {soc_util_efficiency[vm.name]}")

  0%|          | 0/10000 [00:00<?, ?it/s]

Social Utility Efficiency for Plurality: 0.6604769024150335
Social Utility Efficiency for PluralityWRunoff: 0.8237844149350064
Social Utility Efficiency for Instant Runoff: 0.8846071885329869
Social Utility Efficiency for Borda: 0.9802704868067399
Social Utility Efficiency for Coombs: 0.9780310248868681
Social Utility Efficiency for Blacks: 0.9856360516343405
Social Utility Efficiency for Minimax: 0.97972233449008
Social Utility Efficiency for Stable Voting: 0.9854006210048651
Social Utility Efficiency for Beat Path: 0.9796075172450456
Social Utility Efficiency for Ranked Pairs ZT: 0.9849917529535823
